# 1. Librerías & Set Up

👉🏻 Instalaremos todas las librerias necesarias para poder trabajar con PySpark y Neo4j

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c3629350c6cb11056c00b4b9c5366648497ff6da5d4514b2a177876d1d845add
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 4.5 MB/s eta 0:00:00


In [3]:
from pyspark.sql import SparkSession
from itertools import product

In [4]:
import neo4j
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

In [5]:
# Iniciamos una sesion en PySpark
spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# 2. Neo4j Graphs and others examples

👉🏻 Las siguientes funciones permiten cargar grafos de ejemplos para poder realizar las consultas de los patrones de grafo.

👉🏻 Existe la funcion create_neo4j_graph para poder cargar una base de datos en Neo4j y luego obtener las aristas existentes de este grafo.

👉🏻 Existe la funcion load_graph para poder seleccionar el grafo que queremos realizar las consultas.

In [6]:
def load_neo4j_graph():
  """
  Funcion para cargar la base de datos de twitter como ejemplo de grafo en Neo4j.
  """
  # Realizamos la conexion con la base de datos del grafo guardado en Neo4j.
  NEO4J_URI = "neo4j+s://demo.neo4jlabs.com"
  NEO4J_USERNAME = "twitter"
  NEO4J_PASSWORD = "twitter"
  AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

  driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH)
  with driver.session() as session:
      try:
          session.run("RETURN 1")
          print("Connection to Neo4j established successfully!")
      except Exception as e:
          print(f"Failed to connect to Neo4j: {e}")
      # Query que permite cargar todas las aristas de cada usuario
      # (descartamos el usuario Neo4j para tener una cantidad mas variada xd,
      # ya que si lo dejabamos se repetia mucho)
      query = """
      MATCH (u1:User)-[r:INTERACTS_WITH | SIMILAR_TO]->(u2:User)
      WHERE u1.name <> "Neo4j" AND u2.name <> "Neo4j"
      RETURN u1.name as user1, type(r) AS relationship, u2.name AS user2
      """
      result = session.run(query)
      df_edges = pd.DataFrame(result.data())
      relationship = df_edges['relationship'].unique()

      # Definimos una clase para cada relacion ...
      relationship_class = {i : rel for rel, i in enumerate(relationship)}
      for inter, inter_class in relationship_class.items():
          print(f"{inter} AS class {inter_class}")
      df_edges['relationship'].replace(relationship_class, inplace = True)

      # Transformamos cada usuario a un numero (su identificador) en base
      # a su nombre de usuari que debe ser unico ...
      users1 = df_edges['user1'].unique().tolist()
      users2 = df_edges['user2'].unique().tolist()
      users = list(set(users1 + users2))
      users_id = {user : id for id, user in enumerate(users)}
      df_edges['user1'].replace(users_id, inplace = True)
      df_edges['user2'].replace(users_id, inplace = True)

      np.save('users_id', users_id, allow_pickle = True)
      np.save('relationship_class', relationship_class, allow_pickle = True)

      # Cargamos el grafo en una lista de tuplas
      neo4f_graph = [(item[0], item[1], item[2]) for item in df_edges.values]
  driver.close()
  return neo4f_graph


In [7]:
def load_graph(option):
  """
  Funcion que selecciona el grafo de ejemplo, retorna una lista de tuplas
  de las aristas del grafo.
  """
  if option == "neo4j-graph":
    neo4f_graph = load_neo4j_graph()
    return neo4f_graph

  elif option == "basic-example":
    return [
        (1,11,2), (1,11,3), (2,11,3), (3,11,2),
        (3,11,4), (4,11,1), (4,11,2), (4,11,3),
        (4,12,5), (5,12,1), (5,12,2), (5,12,6),
        (1,12,2), (2,12,3), (3,12,1)
    ]
  else:
    print("La opcion no existe :(")
    return None

# 3. MapReduce Algorithm

## 3.1 Implementation for triangles

👉🏻 Las siguientes funciones permiten desarrollar el algoritmo MapReduce implementado con PySpark

In [8]:
def hash(n):
  """
  Returns number mod 2. The ouput will be 0 or 1.
  """
  return n % 2

👉🏻 Las funciones siguientes permiten simular la fase de mapeo del algoritmo.

In [9]:
def get_keys(edge, b_dim, b_set, pattern_dim):
  """
  output: retorna las llaves correspondientes para un vertice.

  Idea general: buscamos el par hash_n1,hash_n2 dentro de las posibles
  combinaciones dentro del espacio de imagenes de la funcion de hash.
  Dentro del for, obtenemos un string con la codificacion de las llaves y luego
  verificamos si es una llave candidata para el vertice entregado:

  1.  sequence_in_reducer: Si la secuencia 'b1b2' esta en la llave del reducer
      codificada como hash(n1)hash(n2) ?, donde ? = 0 o 1, entonces se considerará
      el par reducer_key : edge.
  2.  edge_case: El otro caso, es para cuando tenemos por ejemplo x = n2 y z = n1
      para patrones de 3 vertices.
  """

  hash_n1 = hash(edge[0]) # valor de hash para el nodo 1 = b1
  hash_n2 = hash(edge[2]) # valor de hash para el nodo 2 = b2
  values = [] # posible keys
  sequence = '{}{}'.format(hash_n1, hash_n2)

  for i in range(0, b_dim ** pattern_dim):
    reducer = ''.join(str(num) for num in b_set[i])
    sequence_in_reducer = sequence in reducer
    edge_case = reducer[0] == sequence[1] and reducer[pattern_dim - 1] == sequence[0]
    if sequence_in_reducer or edge_case:
        reducer_key = tuple(int(digit) for digit in reducer)
        values.append((reducer_key, edge))

  return values


def map_phase(rdd, b_dim, b_set, pattern_dim):
  """
  input:
    - rdd: RDD del grafo de dimension 'dim'
    - b_dim: Cantidad de elementos de las imagenes de la funcion de hash.
    - b_set: Imagenes de la funcion de hash.
    - pattern_dim: cantidad de nodos del patron de grafo.
  ouput: Mapeo de cada arista con respecto a las llaves
  """

  mapped_keys = rdd.flatMap(lambda edge: get_keys(edge, b_dim, b_set, pattern_dim))

  return mapped_keys

👉🏻 Las siguientes funciones permiten simular la fase de reduce del algoritmo.

In [10]:
def find_patterns(edges, pattern_dim):
    """
    Funcion que retorna todos los patrones encontrados de la forma
    (n1,...,nl) : [[(n1,label,n2), ..., (nl-1,label,nl)], ... ]
    donde (n1,...,nl) corresponde a los nodos que forman el patron
    y la llave corresponde a los patrones posibles formados por estos nodos
    para una etique
    """
    neighbors = get_neighbors(edges)
    cycles = find_cycles(edges, neighbors, pattern_dim)
    return cycles


def get_neighbors(edges):
    """
    Funcion que retorna todas las aristas consecutivas a otras aristas.
    Es decir, aristas vecinas.
    """
    neighbors = []
    for i in range(len(edges)):
        curr_edge = edges[i]
        neighbors.append([])
        for j in range(len(edges)):
            next_edge = edges[j]
            if next_edge == curr_edge:
                continue
            if curr_edge[2] == next_edge[0]:
                neighbors[i].append(next_edge)
    return neighbors


def find_cycles(edges, neighbors, pattern_dim):
    """
    Funcion que verifica si existe algun grafo ciclico en el grafo de una
    dimension pattern_dim. Retorna todos los ciclos encontrados.
    Utilizamos el algoritmo DFS iterativo para la busqueda de ciclos.
    """
    cycles = [] # Lista de ciclos encontrados

    for edge in edges: # Por cada arista del grafo
        visited = set() # Determinamos las aritas que ya han sido visitadas
        # Inicializamos el stack con con la
        stack = [(edge, [edge])]
        # arista donde empezamos el recorrido

        while len(stack):
            curr_edge, path = stack.pop() # Extraemos la primera arista del stack
            # Recorremos esta arista, por lo que la marcamos como visitada
            visited.add(curr_edge)
            # Caso 1: Si el camino recorrido actual supera la cantidad de nodos del bgp,
            #         descartamos este camino
            if len(path) > pattern_dim:
                continue
            # Caso 2: Si el camino recorrido tiene un largo igual a pattern_dim,
            #         y son aristas transitivas, entonces un posible ciclo.
            first_node = path[0][0] # primer nodo del camino
            last_node = path[-1][2] # ultimo nodo del caminmo
            if len(path) == pattern_dim and first_node == last_node:
                cycles.append(path)
                # Agregamos el camino y seguimos recorriendo
                continue
            # Si no es ciclico el camino desde el nodo actual,
            # entonces iteramos dentro de sus vecinos
            for neighbor_edge in neighbors[edges.index(curr_edge)]:
                if neighbor_edge not in visited:
                    stack.append((neighbor_edge, path + [neighbor_edge]))

    return cycles


def get_nodes(pattern, pattern_dim):
    """
    Funcion que retorna la tupla con los nodos que forman un ciclo.
    """
    nodes = []
    for i in range(pattern_dim):
      edge = pattern[i]
      nodes.append(edge[0])

    return (tuple(sorted(nodes)), tuple(sorted(pattern)))


def get_unique_lists(pattern1, pattern2):
    """
    Funcion que elimina patrones de grafo duplicados para cada llave
    (n1,n2, ..., nl). Si tenemos l = 3, entonces seria (n1,n2,n3)
    """
    lst = []
    if pattern2 != pattern1:
      return lst.append(pattern2)
    else:
      return pattern1


def reduce_phase(mapped_keys, pattern_dim):
    """
    input: RDD del grafo y cantidad de nodos del patron de grafo.
    output: patrones encontrados.
    """
    # Agrupamos por llaves
    reducers = mapped_keys.groupByKey().mapValues(list)
    # Filtramos solamente los grupos conformados por 3 aristas ...
    reducers_edges = reducers.map(lambda v: v[1]) \
                    .filter(lambda x: len(x) >= pattern_dim)
    # Por cada grupo buscamos un patron de grafo
    patterns = reducers_edges \
              .map(lambda edges: find_patterns(edges, pattern_dim)) \
              .flatMap(list) \
              .map(lambda pattern: get_nodes(pattern, pattern_dim)) \
              .groupByKey().mapValues(lambda v: list(set(v)))


    return patterns

👉🏻 Funcion final que simula el algoritmo MapReduce. Recibe los parametros correspondientes del grafo; las dimensiones del conjunto de imagenes de la funcion de hash; el conjunto de imagenes de la funcion de hash; la dimension del patron de grafo.

In [11]:
def map_reduce(rdd_graph, b_dim, b_set, pattern_dim, show=False):
  """
  Funcion que simula el algoritmo MapReduce, en donde se distribuye el grafo
  en diferentes reducers y luego obtenemos los posibles patrones
  de grafos formados combinando las informacion de todos los reducers.
  """
  # Fase de Map: Obtenemos las llaves de cada reducer y el conjunto de aristas mapeados a estas llaves.
  mapped_keys = map_phase(rdd_graph, b_dim, b_set, pattern_dim)

  if show: # Mostras las llaves mapeadas
    print("Fase Map: \n")
    for key in mapped_keys.collect():
      print(key)
    print("\nFase Reduce\n")

  # Fase Reduce: Obtenemos todos los posibles patrones de L nodos.
  patterns = reduce_phase(mapped_keys, pattern_dim)
  return patterns

## 3.2 Implementation for Squares




👉🏻 Las siguientes funciones fueron definidas para realizar las consultas de sobre subgrafos de 4 variables.

In [205]:
def get_all_combinations(edges):
    """
    Funcion que retorna todos los posibles subgrafos de 4 nodos que existen
    dentro de un subgrafo de cada reducer.
    """

    size = len(edges)
    all_patterns = []

    for edge in edges: # iteramos por cada arista
        pattern = [edge] # inicializamos un posible patron de solo 4 nodos
        initial_edge = edge
        # inicializamos los nodos del subgrafo
        nodes = set([initial_edge[0], initial_edge[2]])

        for curr_edge in edges: # iteramos por el resto de las aristas
            if initial_edge == curr_edge:
                continue
            # consideramos los nodos que ya estan dentro del subgrafo
            # mas el nuevo nodo
            new_nodes = nodes.union([curr_edge[0], curr_edge[2]])

            # si al agregar el nuevo nodo no superamos las 4 variables,
            # entonces lo agregamos (junto con su arista respectiva)
            if len(new_nodes) <= 4:
                pattern.append(curr_edge)
                nodes = new_nodes # consideramos el nuevo conjunto de nodos

        if len(nodes) == 4:
            all_patterns.append(pattern)

    return all_patterns


def map_reduce_squares(rdd_graph, b_dim, b_set, pattern_dim):
    """
    Funcion que simula el algoritmo Map Reduce para subgrafos de solo 4 nodos.
    Retorna todos los posibles subgrafos del grafo entregado en la RDD
    con solo 4 nodos.
    """
    # Realizamos el Map Phase para obtener todos los pares llaves - valor
    mapped_keys = map_phase(rdd_graph, b_dim, b_set, pattern_dim)
    # Reducimos los valores dentro de cada reducer, donde solo consideramos
    # algun grupo que tenga al menos 3 aristas (posible patron de 4 variables)
    reducers = mapped_keys.groupByKey().mapValues(list)
    # Dentro de cada reducer, formamos todos los posibles patrones de solo
    # 4 variables distintas
    all_combinations = reducers.mapValues(get_all_combinations)
    # filtramos las llaves que tienen como valores nulos (subgrafos con dimension menor a 4 )
    all_combinations = all_combinations.filter(lambda x: x[1] != [])

    return all_combinations


def preprocess_query(query):
    """
    Funcion que retorna la consulta en una lista de tuplas
    ingresada en get_all_matches.
    """
    processed_query = query.replace(" ", ";").replace("(", "").replace(")", "").split(",;")
    processed_query = [tuple(item.split(",")) for item in processed_query]
    return processed_query


def create_matrix_query(preprocessed_query):
    """
    Funcion que retorna la matriz de consulta.
    A: variables
    L: labels
    """
    A, L = [], []
    for edge in preprocessed_query:
        if edge[0] not in A:
            A.append(edge[0])
        if edge[2] not in A:
            A.append(edge[2])
        if int(edge[1]) not in L:
            L.append(int(edge[1]))

    # asociamos cada variable/label un indice
    dict_var_dir = {var: dir for dir, var in enumerate(A)}
    dict_label_dir = {label: dir for dir, label in enumerate(L)}
    M = np.zeros((len(A), len(L), len(A))) # matriz de consulta de dimension |A|x|L|x|A|
    # Rellenamos la matriz M con solo 1's en las posiciones donde exista una arista
    for edge in preprocessed_query:
        node_1 = edge[0] # primer nodo n1
        label = int(edge[1]) # label n1 -[label]-> n2
        node_2 = edge[2] # segundo nodo n2
        dir_node_1 = dict_var_dir[node_1]
        dir_label = dict_label_dir[label]
        dir_node_2 = dict_var_dir[node_2]

        M[dir_node_1][dir_label][dir_node_2] = 1

    return M

def matches(subgraphs, matrix_query):
    """
    Funcion que retorna los subgrafos que realizan match con
    la matriz de consulta (patron de grafo)
    """
    results = []
    # Iteramos por todo el subrgrafo y verificamos las posibles contenidas
    # que hagan match con la matriz de consulta
    for subgraph in subgraphs: # para cada subgrafo de la llabe
        match_found = False
        for i in range(len(subgraph)):
            if match_found:
                break
            # hacemos un desplazamiento de aristas
            shift_subgraph = subgraph[i:] + subgraph[:i]
            # obtenemos la matriz correspondiente a este subgrafo
            matrix_subgraph = create_matrix_query(shift_subgraph)
            # si hacen match, entonces lo agregamos y seguimos viendo otro subgrafo
            if np.array_equal(matrix_query, matrix_subgraph):
                results.append(subgraph)
                match_found = True

    return results

def generate_nodes(subgraph):
    """
    Funcion que retorna una tupla de los nodos de un subgrafo.
    """
    return tuple(set(edge[0] for edge in subgraph).union(edge[2] for edge in subgraph))


def get_all_matches(rdd_graph, query, show=False, more_info=False):
    """
    Funcion que retorna todos los matches con respecto a una consulta de
    solo 4 variables
    """
    # Primero obtenemos todos los posibles subgrafos de 4 nodos, independiente
    # de la cantidad de aristas
    b_sq = 2
    l_sq = 4
    b_set_0 = list(range(b_sq))
    reducers_sq = list(product(b_set_0, repeat=l_sq))

    # obtenemos todos los subgrafos de 4 nodos con MapReduce
    all_subgraphs = map_reduce_squares(rdd_graph, b_sq, reducers_sq, l_sq)
    preprocessed_query = preprocess_query(query)
    # matriz de consulta codificada
    matrix_query = create_matrix_query(preprocessed_query)
    # todos los patrones de grafos que hacen match
    patterns_result = all_subgraphs \
                        .mapValues(lambda values: matches(values, matrix_query))\
                        .filter(lambda v: v[1] != []).values().flatMap(list)
    if more_info:
        return patterns_result.map(lambda x: (generate_nodes(x), x))

    result = patterns_result.map(lambda x: generate_nodes(x)).distinct()

    if show:
        for k, vs in all_subgraphs.collect():
            for v in vs:
                print(v)

    return result.collect()

# 4. Aplicacion MapReduce en Ejemplo Basico

👉🏻 En esta seccion aplicaremos el algoritmo MapReduce para un grafo de prueba pequeño (el que fue dado al inicio de la tarea, con algunas modificaciones). Este lo guardamos como 'basic-example'.

In [190]:
graph = load_graph('basic-example')
rdd_graph = sc.parallelize(graph)
rdd_graph.collect()

[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3),
 (4, 12, 5),
 (5, 12, 1),
 (5, 12, 2),
 (5, 12, 6),
 (1, 12, 2),
 (2, 12, 3),
 (3, 12, 1)]

## 4.1 MapReduce Algorithm for Triangles

👉🏻 Para encontrar triangulos dentro de un grafo utilizaremos el algoritmo Map Reduce para parametros b_0 = 2 y l = 3.

👉🏻 En este caso, b_0=2 ya que al utilizar la funcion de hash modular `f(x) = x mod 2`, generamos un conjunto de 2 imagenes: {0,1}. En consecuencia, los posibles reducers son:
```markdown
REDUCERS = [(0, 0, 0),
            (0, 0, 1),
            (0, 1, 0),
            (0, 1, 1),
            (1, 0, 0),
            (1, 0, 1),
            (1, 1, 0),
            (1, 1, 1)]
```

👉🏻 Dado que estamos buscando subgrafos de 3 nodos, entonces l_0 = 3.

👉🏻 Las aristas se distribuyen dentro de los reducers, por lo que almacenamos el grafo de manera distribuida.En cada reducer verificamos si las aristas generan triangulos, en el caso que si (formaria un ciclo) se almacenará en una lista y retornaremos los nodos correspondientes.




In [ ]:
# Dimension de elementos del conjunto de imagenes de la funcion de hash: |{0,1}|
b_0 = 2
# Dimension del patron de grafo (triangulo para este caso)
l_0 = 3
# Conjunto de imagenes de la funcion de hash
b_set_0 = list(range(b_0))
# Posibles reducers
reducers_0 = list(product(b_set_0, repeat=l_0))

patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0)

patterns.collect()

[((2, 3, 4),
  [((2, 11, 3), (3, 11, 4), (4, 11, 2)),
   ((2, 12, 3), (3, 11, 4), (4, 11, 2))]),
 ((1, 2, 3),
  [((1, 12, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 11, 3), (3, 12, 1)),
   ((1, 11, 2), (2, 12, 3), (3, 12, 1)),
   ((1, 12, 2), (2, 11, 3), (3, 12, 1))]),
 ((1, 3, 4), [((1, 11, 3), (3, 11, 4), (4, 11, 1))])]

👉🏻 Al aplicar el algoritmo mapReduce, podemos notar que existen 3 tripletas de nodos que forman un triangulo. En donde estos nodos permiten formar diferentes patrones de grafos triangulares. En este caso, como tenemos mas de una etiqueta distinta, podemos notar que para una tripleta de nodo, forman mas de un triangulo.
```markdown
  (2,3,4); (1,2,3); (1,3,4)
```

👉🏻 Las siguientes funciones se definiran con el fin de desarrollar consultas sobre estos grafos.


In [ ]:
def filter_patterns(pattern, pattern_dim, labels):
  """
  Funcion que retorna los patrones filtrados segun el label proporcionado como
  parametro.
  """
  pattern_labels = [edge[1] for edge in pattern]
  for i in range(pattern_dim + 1):
      # verificamos si coinciden los labels ...
      if pattern_labels == labels:
        return pattern
      # Hacemos un desplazamiento hacia la derecha ...
      pattern_labels = pattern_labels[i:] + pattern_labels[:i]

  return None

def bgp_query(rdd_graph, b_dim, reducers, pattern_dim, query):
  """
  Funcion que retorna todos los patrones de grafos de dimension "pattern_dim"
  que coincidan segun la query entregada.
  """

  patterns = map_reduce(rdd_graph, b_dim, reducers, pattern_dim).values().flatMap(list)
  processed_query = query.replace("(", "").replace(")", "").split(",")
  labels = [int(part.strip()) for part in processed_query if part.strip().isdigit()]

  result = patterns.filter(lambda pattern: filter_patterns(pattern, pattern_dim, labels))
  return result

In [ ]:
# Realizamos la siguiente consulta en donde queremos obtener todos los triangulos
# donde la etiqueta es 11, independiente del nodo.
result = bgp_query(rdd_graph, b_0, reducers_0, l_0, "(x,11,y), (y,11,z), (z,11,w)")
for res in result.collect():
  print(res)

((2, 11, 3), (3, 11, 4), (4, 11, 2))
((1, 11, 3), (3, 11, 4), (4, 11, 1))


## 4.2 MapReduce Algorithm for Squares


👉🏻 Ahora aplicaremos nuestro algoritmo para la busqueda de patrones de grafos de 4 variables. En primer lugar, utilizaremos el algoritmo Map Reduce definido para patrones de grafo de solo 4 variables definido en la **seccion 3.2**.
Esta funcion utiliza retorna todos los patrones de 4 variables que hagan match con la consulta definida en la funcion.

In [193]:
# A continuacion mostraremos todos los subgrafos posibles de solo 4 nodos,
# independiente de las aristas que tengan
get_all_matches(rdd_graph, "(x,12,y), (z,11,w), (x,12,z), (x,12,w), (z,12,w)", show=True)

[(1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 1), (4, 11, 2), (4, 11, 3), (1, 12, 2), (2, 12, 3), (3, 12, 1)]
[(1, 11, 3), (1, 11, 2), (2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 1), (4, 11, 2), (4, 11, 3), (1, 12, 2), (2, 12, 3), (3, 12, 1)]
[(2, 11, 3), (1, 11, 2), (1, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 1), (4, 11, 2), (4, 11, 3), (1, 12, 2), (2, 12, 3), (3, 12, 1)]
[(3, 11, 2), (1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 4), (4, 11, 1), (4, 11, 2), (4, 11, 3), (1, 12, 2), (2, 12, 3), (3, 12, 1)]
[(3, 11, 4), (1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 2), (4, 11, 1), (4, 11, 2), (4, 11, 3), (1, 12, 2), (2, 12, 3), (3, 12, 1)]
[(4, 11, 1), (1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 2), (4, 11, 3), (1, 12, 2), (2, 12, 3), (3, 12, 1)]
[(4, 11, 2), (1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 2), (3, 11, 4), (4, 11, 1), (4, 11, 3), (1, 12, 2), (2, 12, 3), (3, 12, 1)]
[(4, 11, 3), (1, 11, 2), (1, 11, 3), (2, 11, 3), (3, 11, 2), (3, 11, 

[(1, 2, 5, 6)]

In [191]:
# Ahora realizaremos la consulta del siguiente subgrafo
get_all_matches(rdd_graph, "(x,12,y), (z,11,w), (x,12,z), (x,12,w), (z,12,w)")

[(1, 2, 5, 6)]

# 5. Aplicacion de un grafo a gran escala: Twitter Graph

👉🏻 Ahora cargaremos un grafo de mayor tamaño (700 aristas) para evaluar nuestro algoritmo. El siguiente grafo corresponde a un grafo que modela la interaccion de cada usuario en la red social twitter.
El grafo sera cargado como lista de tuplas, en donde cada tupla modela la relacion entre un usuario y otro. Consideramos las siguientes 2 tipos de relaciones

    Relacion 0: INTERACTS_WITH - El usuario 1 interacciona con otro usuario 2
    Relacion 1: SIMILAR_TO - El usuario 1 es similar al usuario 2

Referencia: https://github.com/neo4j-graph-examples/twitter-v2


In [196]:
graph = load_graph('neo4j-graph')
rdd_graph = sc.parallelize(graph)

Connection to Neo4j established successfully!
INTERACTS_WITH AS class 0
SIMILAR_TO AS class 1


In [197]:
rdd_graph.count() # Cantidad de aristas del grafo

744

## 4.1 MapReduce Algorithm for Triangles

👉🏻 Ahora aplicaremos MapReduce para encontrar los patrones de grafos con forma de triangulo dentro del grafo cargado desde Neo4j.



In [ ]:
patterns = map_reduce(rdd_graph, b_0, reducers_0, l_0)
patterns.collect()

[((294, 430, 499), [((294, 0, 430), (430, 1, 499), (499, 1, 294))]),
 ((94, 118, 403), [((94, 1, 403), (118, 0, 94), (403, 0, 118))]),
 ((178, 187, 448),
  [((178, 0, 448), (187, 0, 178), (448, 1, 187)),
   ((178, 1, 187), (187, 0, 448), (448, 1, 178))]),
 ((398, 458, 459),
  [((398, 1, 458), (458, 1, 459), (459, 0, 398)),
   ((398, 1, 459), (458, 0, 398), (459, 0, 458))]),
 ((294, 327, 430),
  [((294, 0, 327), (327, 0, 430), (430, 1, 294)),
   ((294, 0, 430), (327, 1, 294), (430, 1, 327))]),
 ((42, 395, 430), [((42, 1, 430), (395, 1, 42), (430, 0, 395))]),
 ((115, 430, 452), [((115, 1, 452), (430, 0, 115), (452, 1, 430))]),
 ((42, 115, 430), [((42, 1, 430), (115, 1, 42), (430, 0, 115))]),
 ((42, 243, 452),
  [((42, 0, 243), (243, 0, 452), (452, 1, 42)),
   ((42, 0, 452), (243, 1, 42), (452, 1, 243))]),
 ((42, 115, 452), [((42, 0, 115), (115, 1, 452), (452, 1, 42))]),
 ((42, 395, 452), [((42, 0, 452), (395, 1, 42), (452, 1, 395))]),
 ((42, 156, 395),
  [((42, 0, 395), (156, 1, 42), (39

👉🏻 El algoritmo es bien rapido de ejecutar, y podemos ver claramente las relaciones entre diferentes usuarios, para diferentes labels.

👉🏻 Para poder analizar las relaciones y compararlo con el grafo mostrado en Neo4j, realizaremos una consulta, y luego compararemos los resultados ...

In [198]:
# Cargamos el diccionario que guardamos los nombres de usuarios y el id asociado.
users_id = np.load('users_id.npy', allow_pickle=True).item()
dict_users_id = {value : key for key, value in users_id.items()}
# Cargamos el tipo de relacion.
relationship_class = np.load('relationship_class.npy', allow_pickle=True).item()
dict_relation_class = {value : key for key, value in relationship_class.items()}

In [ ]:
result = bgp_query(rdd_graph, b_0, reducers_0, l_0, "(x,1,y), (y,0,z), (z,0,x)")
result.collect()

[((94, 1, 403), (118, 0, 94), (403, 0, 118)),
 ((398, 1, 459), (458, 0, 398), (459, 0, 458)),
 ((42, 0, 395), (156, 1, 42), (395, 0, 156)),
 ((42, 0, 243), (156, 1, 42), (243, 0, 156)),
 ((156, 1, 430), (395, 0, 156), (430, 0, 395)),
 ((42, 0, 156), (115, 1, 42), (156, 0, 115)),
 ((239, 1, 501), (269, 0, 239), (501, 0, 269)),
 ((243, 1, 327), (327, 0, 395), (395, 0, 243)),
 ((115, 1, 395), (243, 0, 115), (395, 0, 243)),
 ((403, 0, 470), (413, 1, 403), (470, 0, 413)),
 ((40, 0, 273), (273, 1, 341), (341, 0, 40)),
 ((327, 0, 499), (430, 1, 327), (499, 0, 430)),
 ((156, 1, 327), (327, 0, 395), (395, 0, 156)),
 ((73, 0, 110), (110, 1, 327), (327, 0, 73)),
 ((73, 1, 327), (327, 0, 430), (430, 0, 73)),
 ((156, 1, 395), (243, 0, 156), (395, 0, 243)),
 ((42, 0, 395), (115, 1, 42), (395, 0, 115)),
 ((327, 0, 430), (395, 1, 327), (430, 0, 395)),
 ((42, 0, 395), (243, 1, 42), (395, 0, 243)),
 ((115, 1, 243), (243, 0, 452), (452, 0, 115)),
 ((115, 1, 243), (156, 0, 115), (243, 0, 156))]

In [199]:
def load_usernames(pat):
  """
  Funcion que retorna el nombre de usuario codificado a un id, y tambien transforma
  la clase del label a su valor original
  """
  list_users = []
  for i in range(len(pat)):
    edge = pat[i]
    user1 = dict_users_id[edge[0]]
    label = dict_relation_class[edge[1]]
    user2 = dict_users_id[edge[2]]
    list_users.append((user1, label, user2))
  return list_users

👉🏻 Ahora con la consulta anterior, analizaremos los usuarios involucrados en el siguiente caso: el primer usuario interacciona con el segundo; el segundo es similar al tercero; y el tercero el similar al primero.

In [ ]:
result.map(load_usernames).collect()

[[('CRYPTEMES', 'SIMILAR_TO', 'Michael Hunger'),
  ('Sandro Miccoli', 'INTERACTS_WITH', 'CRYPTEMES'),
  ('Michael Hunger', 'INTERACTS_WITH', 'Sandro Miccoli')],
 [('Mahendrawathi ER', 'SIMILAR_TO', 'Nur Aini Rakhmawati'),
  ('Radit - Pendekar Syair Berbusa', 'INTERACTS_WITH', 'Mahendrawathi ER'),
  ('Nur Aini Rakhmawati', 'INTERACTS_WITH', 'Radit - Pendekar Syair Berbusa')],
 [('Born To Be Idle', 'INTERACTS_WITH', 'John Wolpert'),
  ('Brana Rakic', 'SIMILAR_TO', 'Born To Be Idle'),
  ('John Wolpert', 'INTERACTS_WITH', 'Brana Rakic')],
 [('Born To Be Idle', 'INTERACTS_WITH', 'Chill Zone'),
  ('Brana Rakic', 'SIMILAR_TO', 'Born To Be Idle'),
  ('Chill Zone', 'INTERACTS_WITH', 'Brana Rakic')],
 [('Brana Rakic', 'SIMILAR_TO', 'Nancy Tremblay❤😀🌻🇨🇦'),
  ('John Wolpert', 'INTERACTS_WITH', 'Brana Rakic'),
  ('Nancy Tremblay❤😀🌻🇨🇦', 'INTERACTS_WITH', 'John Wolpert')],
 [('Born To Be Idle', 'INTERACTS_WITH', 'Brana Rakic'),
  ('guinnessstache', 'SIMILAR_TO', 'Born To Be Idle'),
  ('Brana Rakic', 

## 4.2 MapReduce For Squares

👉🏻 Ahora aplicaremos el algoritmo MapReduce para buscar patrones todos los posibles subgrafos de solo 4 variables, para luego realizar una consulta.

In [209]:
# Primer ejemplo: usuario 1 interacciona con usuario 2 y usuario 3 interacciona con usuario 4.
get_all_matches(rdd_graph, "(x,0,y), (z,0,w)")

[(465, 282, 429, 390),
 (282, 21, 429, 390),
 (282, 429, 390, 135),
 (282, 429, 390, 31),
 (282, 475, 429, 390),
 (282, 211, 429, 390),
 (282, 29, 429, 390),
 (282, 483, 429, 390),
 (130, 339, 429, 390),
 (249, 130, 429, 390),
 (253, 130, 429, 390),
 (130, 291, 429, 390),
 (130, 195, 429, 390),
 (130, 45, 429, 390),
 (308, 429, 390, 391),
 (441, 404, 429, 390),
 (390, 341, 382, 429),
 (292, 341, 390, 429),
 (42, 341, 390, 429),
 (336, 341, 390, 429),
 (498, 341, 390, 429),
 (390, 341, 190, 429),
 (252, 341, 390, 429),
 (390, 341, 134, 429),
 (492, 341, 390, 429),
 (4, 341, 390, 429),
 (412, 341, 390, 429),
 (460, 341, 390, 429),
 (112, 341, 390, 429),
 (390, 341, 214, 429),
 (328, 341, 390, 429),
 (184, 341, 390, 429),
 (200, 259, 429, 390),
 (259, 460, 429, 390),
 (504, 3, 429, 390),
 (504, 429, 390, 383),
 (504, 429, 390, 431),
 (504, 243, 429, 390),
 (504, 451, 429, 390),
 (323, 12, 429, 390),
 (323, 196, 429, 390),
 (13, 390, 429, 430),
 (390, 275, 429, 430),
 (390, 429, 430, 311),

In [212]:
# ahora veremos los usuarios involucrados ...
repeated = []
for nodes, subgraph in get_all_matches(rdd_graph, "(x,0,y), (z,0,w)", more_info=True).collect():
    if subgraph not in repeated:
        repeated.append(subgraph)
        print(load_usernames(subgraph))

[('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn'), ('Michael Simons', 'INTERACTS_WITH', 'C.J.')]
[('Michael Simons', 'INTERACTS_WITH', 'C.J.'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Simons', 'INTERACTS_WITH', 'Cloud Foundry'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Simons', 'INTERACTS_WITH', 'SpringCentral'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Simons', 'INTERACTS_WITH', 'Gerrit Meier'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Simons', 'INTERACTS_WITH', 'Samuel Nitsche'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Simons', 'INTERACTS_WITH', 'Excalidraw'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Simons', 'INTERACTS_WITH', 'Alexey Grigorev'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]
[('Michael Simons', 'INTERACTS_WITH', 'Spring Data Team'), ('Kesavan Nair (Kay)', 'INTERACTS_WITH', 'Donny Flynn')]

In [228]:

get_all_matches(rdd_graph, "(x,1,y), (z,0,w), (y,0,x)")

[(282, 341, 429, 390),
 (377, 282, 429, 390),
 (282, 429, 390, 207),
 (229, 282, 429, 390),
 (130, 429, 390, 303),
 (130, 429, 390, 79),
 (130, 341, 429, 390),
 (504, 341, 390, 429),
 (396, 341, 390, 429),
 (194, 341, 390, 429),
 (282, 341, 390, 429),
 (130, 341, 390, 429),
 (504, 259, 429, 390),
 (504, 341, 429, 390),
 (377, 390, 429, 430),
 (100, 429, 390, 63),
 (283, 100, 429, 390),
 (194, 341, 429, 390),
 (194, 429, 390, 119),
 (280, 221, 429, 390),
 (248, 429, 390, 295),
 (280, 221, 390, 429),
 (459, 428, 429, 390),
 (428, 429, 390, 79),
 (64, 429, 390, 79),
 (58, 429, 390, 231),
 (153, 390, 429, 54),
 (170, 429, 390, 79),
 (152, 461, 429, 390),
 (32, 429, 390, 437),
 (170, 429, 390, 437),
 (170, 390, 429, 389),
 (390, 429, 326, 437),
 (464, 429, 390, 437),
 (120, 437, 390, 429),
 (32, 437, 390, 429),
 (390, 437, 326, 429),
 (464, 437, 390, 429),
 (170, 437, 390, 429),
 (120, 429, 390, 437),
 (170, 389, 390, 429),
 (474, 85, 429, 390),
 (152, 461, 390, 429),
 (488, 281, 429, 390),

In [234]:
get_all_matches(rdd_graph, "(x,0,y), (w,0,z), (w,0,x), (x,1,w)")

[(356, 282, 180, 430), (356, 282, 50, 430)]